In [ ]:
{
    var koa = require('koa');
    var app = new koa();
    app.use(async ctx=>{
        ctx.body='hello';
    })
    
    app.listen(process.argv[2]);
}

In [ ]:
{
    var koa = require('koa');
    var app = new koa();
    var port = process.argv[2];
    
    app.use(async (ctx,next)=>{
        if(ctx.path=='/')
            ctx.body='hello koa';
        else
           await next();
    })
    
    app.use(async (ctx,next)=>{
        if(ctx.path=='/400')
            ctx.body ='page not found';
        else
           await next();
    })
    
    app.use(async (ctx,next)=>{
        if(ctx.path=='/500')
            ctx.body = 'internal server error';
        else
           await next();
    })
    app.listen(port);
}

In [ ]:
{
    var koa = require('koa');
    var app = koa();
    app.use(function* (next){
        if(this.path !=='/'){
            return yield next;
        }
        this.body = 'hello koa';
    })
    app.use(function* (next){
        if(this.path!=='/404'){
            return yield next;
        }
        this.body = 'page not found';
    })
    app.use(function* (next){
        if(this.path!=='/500'){
            return yield next;
        }
        this.body = 'internal server error';
    })
    
    app.listen(process.argv[2]);
}

In [ ]:
{
    var koa = require('koa');                             
    var parse = require('co-body');                       

    var app = koa();                                      

    app.use(function* (next) {                            
      // only accept POST request                         
      if (this.method !== 'POST') return yield next;      

      // max body size limit to `1kb`                     
      var body = yield parse(this, { limit: '1kb' });     

      // if body.name not exist, respond `400`            
      if (!body.name) this.throw(400, '.name required');  

      this.body = body.name.toUpperCase();                
    });                                                   

    app.listen(process.argv[2]);                          
}

In [ ]:
{
    var koa = require('koa');
    var parse = require('co-body');
    var app = new koa();

    var port = process.argv[2];

    app.use(async (ctx,next)=>{
            if(ctx.method!=='POST')
               return await next();
            var body = await parse(ctx);
            if(body.name){
                    ctx.body = body.name.toUpperCase();
            }else{
                   await next();
            }
            //console.log(JSON.stringify(body));
    })

    app.use(async (ctx,next) => {
            if(ctx.path=='/')
                    ctx.body = 'hello koa';
            else
                  await  next();
    })

    app.use(async (ctx,next) => {
            if(ctx.path=='/404')
                    ctx.body = 'page not found';
            else
                  await  next();
    })

    app.use(async (ctx,next) => {
            if(ctx.path=='/500')
                    ctx.body = 'internal server error';
            else
                  await  next();
    })


    app.listen(port);
}

In [ ]:
{
    const Koa = require('koa');
    const fs = require('fs');

    const app = new Koa();

    app.use(async (ctx, next) => {
      if (ctx.path !== '/json') {
        return await next();
      }

      ctx.body = { foo: 'bar' };
    });

    app.use(async (ctx, next) => {
      if (ctx.path !== '/stream') {
        return await next();
      }

      ctx.body = fs.createReadStream(process.argv[3]);
    });

    app.listen(process.argv[2]);
}

In [ ]:
{
    var koa = require('koa');
    var fs = require('fs');

    var app  = new koa();

    app.use(async (ctx,next)=>{
            if(ctx.is('json')){
                    ctx.body = {message:'hi!'};
                    //ctx.type="json";
            }else {
                    //ctx.type="stream";
                    ctx.body = 'ok';
            }

    })

    app.listen(process.argv[2]);
}

In [ ]:
{
    const Koa = require('koa');

    const app = new Koa();

    app.use(responseTime());
    app.use(upperCase());

    app.use(async ctx => {
      // step 3: respond `hello koa`
      ctx.body = 'hello koa';
    });

    function responseTime() {
      return async (ctx, next) => {
        // step 1: record start time
        const start = new Date;
        await next();
        // step 5: set X-Response-Time head
        ctx.set('X-Response-Time', new Date - start);
      };
    }

    function upperCase() {
      return async (ctx, next) => {
        // step 2: do nothing here
        await next();
        // step 4: convert ctx.body to upper case
        ctx.body = ctx.body.toUpperCase();
      };
    }

    app.listen(process.argv[2]);
}

In [ ]:
{
    const Koa = require('koa');

    const app = new Koa();

    app.use(errorHandler());

    app.use(async ctx => {
      if (ctx.path === '/error') throw new Error('ooops');
      ctx.body = 'OK';
    });

    function errorHandler() {
      return async (ctx, next) => {
        // we catch all downstream errors here
        try {
          await next();
        } catch (err) {
          // set response status
          ctx.status = 500;
          // set response body
          ctx.body = 'internal server error';
          // can emit on app for log
          // app.emit('error', err, ctx);
        }
      };
    }

    app.listen(process.argv[2]);
}

In [ ]:
{
    var koa = require('koa');
    var app  = new koa();
    app.keys = ['secret','keys'];

    app.use(async ctx=>{
            if(ctx.cookies.get('view',{signed:true})){
                    let num = +ctx.cookies.get('view',{signed:true});
                    num++;
                    ctx.cookies.set('view',num,{signed:true});
                    ctx.body = num+' views';
            }else{
                    ctx.cookies.set('view','1',{signed:true});
                    ctx.body = '1 views';
            }
    })

    app.listen(process.argv[2]);
}

In [ ]:
{
    var koa = require('koa');
    var session = require('koa-session');

    var app = new koa();
    app.keys = ['secret','keys'];

    app.use(session(app));

    app.use(async ctx=>{
            let n = ctx.session.views || 0;
            ctx.session.views = ++n;
            ctx.body = n+' views';
    })

    app.listen(process.argv[2]);
}

In [ ]:
{
    var koa = require('koa');
    var views = require('co-views');

    var app = new koa();

    var render = views(__dirname + '/views',{
            ext:'ejs'
    });

    var user = {
            name:{
                    first:'Tobi',
                    last:'Holowaychuk'
            },
            species:'ferret',
            age:3
    };

    app.use(async ctx=>{
            ctx.body = await render('user',{user:user});
    })

    app.listen(process.argv[2]);
}

In [ ]:
{
    const Koa = require('koa');
    const parse = require('co-body');
    const session = require('koa-session');

    const form = '<form action="/login" method="POST">\
      <input name="username" type="text" value="username">\
      <input name="password" type="password" placeholder="The password is \'password\'">\
      <button type="submit">Submit</button>\
    </form>';

    const app = new Koa();

    // use koa-session somewhere at the top of the app
    // we need to set the `.keys` for signed cookies and the cookie-session module
    app.keys = ['secret1', 'secret2', 'secret3'];
    app.use(session(app));

    /**
     * If `ctx.session.authenticated` exist, user will see 'hello world'
     * otherwise, people will get a `401` error  because they aren't logged in
     */
    app.use(async (ctx, next) => {
      if (ctx.path !== '/') return await next();
      if (ctx.session.authenticated) return ctx.body = 'hello world';
      ctx.status = 401;
    });

    /**
     * If successful, the logged in user should be redirected to `/`.
     * hint: use `ctx.redirect`
     */
    app.use(async (ctx, next) => {
      if (ctx.path !== '/login') return await next();
      if (ctx.method === 'GET') return ctx.body = form;
      if (ctx.method !== 'POST') return;

      const body = await parse(ctx);
      const {username, password} = body;
      if (username !== 'username' || password !== 'password') return ctx.status = 400;

      ctx.session.authenticated = true;
      ctx.redirect('/');
    });

    /**
     * Let's redirect to `/login` after every response.
     * If a user hits `/logout` when they're already logged out,
     * let's not consider that an error and rather a "success".
     */
    app.use(async (ctx, next) => {
      if (ctx.path !== '/logout') return await next();
      ctx.session.authenticated = false;
      ctx.redirect('/login');
    });

    app.listen(process.argv[2]);
}